# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118493e+02     1.672261e+00
 * time: 0.12157082557678223
     1     1.086840e+01     1.030734e+00
 * time: 0.9075078964233398
     2    -1.120187e+01     1.266984e+00
 * time: 1.0348658561706543
     3    -3.386164e+01     8.596595e-01
 * time: 1.1929190158843994
     4    -4.724432e+01     6.342808e-01
 * time: 1.3384268283843994
     5    -5.690118e+01     2.112987e-01
 * time: 1.478529930114746
     6    -5.980773e+01     1.314101e-01
 * time: 1.580185890197754
     7    -6.088736e+01     5.430036e-02
 * time: 1.711488962173462
     8    -6.129180e+01     5.791020e-02
 * time: 1.8151438236236572
     9    -6.156366e+01     3.068273e-02
 * time: 1.923987865447998
    10    -6.178087e+01     2.773081e-02
 * time: 2.026516914367676
    11    -6.194978e+01     3.464442e-02
 * time: 2.135725975036621
    12    -6.202845e+01     2.652002e-02
 * time: 2.238101005554199
    13    -6.210770e+01     1.977645e-02
 * time: 2.347611904144287


In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557710
    AtomicNonlocal      14.8522677
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336821

    total               -62.261666460219
